In [181]:
import torch
from einops import einsum, rearrange
from jaxtyping import Float
import math

Máme vsutpní (... seq d_k) tensor

Máme pozice (... seq) tensor

hledáme v podstatě $\theta_{i,k} = \dfrac{i}{\Theta^{2k/d}}$ v paperu je (k-1), $i$ je pozice tokenu $\Theta = 10000$

$ \begin{bmatrix} cos(\beta) & -sin(\beta) \\ sin(\beta) & cos(\beta) \end{bmatrix} $

1. Jako první bych si měl předpočítat tensor d_k/2 2 2 tensor, který bude obsahovat úhly $\dfrac{1}{\Theta^{2k/d}}$
2. Potom reshapnout vstupní tensor na (... seq d_k/2 1 2)
3. Tensor s úhly předělat na ... seq d_k/2 2 2 tensor pomocí pozicí ... seq 1 1 1
4. zavolat sinus cosinus a -sinus podle potřeby
5. vynásobím ... seq dk/2 2 2 s ... seq dk/2 1 2 -> ... seq dk 

Vstup: b=3,seq 2, d_k 6
Výstup: b=3,seq 2

In [190]:
b = 3
seq = 2
d_k = 6
theta = 10000
X = torch.randn(b, seq, d_k)
position = torch.arange(start=1, end=seq + 1, step=1, dtype=torch.float)


In [183]:
X.shape,position.shape

(torch.Size([3, 2, 6]), torch.Size([3, 2]))

In [187]:
Rs = []
for k in range(1,int(d_k / 2) + 1,1):
    Rs.append(1 / math.pow(theta, 2 * k / d_k))
R = torch.tensor(Rs,dtype=torch.float)
R

tensor([4.6416e-02, 2.1544e-03, 1.0000e-04])

## Pure torch

In [191]:
X_preped = rearrange(X,"... seq ( dk_2 x ) -> ... seq dk_2 1 x ",x=2)
X_preped.shape

torch.Size([3, 2, 3, 1, 2])

In [192]:
position_preped = rearrange(position, " ... -> ... 1")
R_preped = position_preped * R
R_preped

tensor([[4.6416e-02, 2.1544e-03, 1.0000e-04],
        [9.2832e-02, 4.3089e-03, 2.0000e-04]])

In [202]:
R_cos = torch.cos(R_preped).unsqueeze(-1)
R_sin = torch.sin(R_preped).unsqueeze(-1)
R_sin,R_cos

(tensor([[[4.6399e-02],
          [2.1544e-03],
          [1.0000e-04]],
 
         [[9.2698e-02],
          [4.3089e-03],
          [2.0000e-04]]]),
 tensor([[[0.9989],
          [1.0000],
          [1.0000]],
 
         [[0.9957],
          [1.0000],
          [1.0000]]]))

In [206]:
R_cos[0,0],-R_sin[0,0],R_sin[0,0],R_cos[0,0]

(tensor([0.9989]), tensor([-0.0464]), tensor([0.0464]), tensor([0.9989]))

In [216]:
R_final = rearrange(torch.stack([R_cos,-R_sin,R_sin,R_cos],dim=-1).squeeze(-2)," ... seq dk_2 (x1 x2) -> ... seq dk_2 x1 x2",x1=2,x2=2)
R_final.shape

torch.Size([2, 3, 2, 2])

In [218]:
preres = einsum(X_preped,R_final," ... seq dk_2 x1 y, ... seq dk_2 x y -> ... seq dk_2 x1 y")
preres.shape

torch.Size([3, 2, 3, 1, 2])

In [220]:
res = rearrange(preres,"... seq dk_2 x y -> ... seq (dk_2 x y)")
res

tensor([[[ 9.6661e-01, -2.4718e-02, -5.4635e-01,  3.4806e-03, -1.6450e+00,
           1.5395e+00],
         [ 8.4087e-04, -1.0848e+00,  2.1471e-02,  1.7415e+00,  7.0707e-02,
           1.8446e-01]],

        [[ 1.9648e+00, -1.5071e+00,  1.1999e+00,  1.4236e+00,  6.1655e-01,
           8.1411e-01],
         [ 1.2184e+00,  9.3472e-01, -4.8975e-01,  3.1724e-01, -4.5013e-01,
           4.9193e-01]],

        [[ 6.8069e-01,  1.3673e+00,  9.1306e-01, -1.0062e-02,  5.7589e-01,
           2.9981e-01],
         [ 5.3776e-01, -1.8274e+00,  1.2193e+00, -1.1139e+00,  2.1593e-03,
           1.2228e+00]]])